## GGRF Accounting Analysis
* Goal: Narrow down projects from Cycle 3 and beyond that need intervention with allocating funds from GGRF.
* Projects with a remaining allocation of $400,000 or more needs intervention.


In [1]:
import A1_data_prep
import A7_accounting_analysis
import pandas as pd
from calitp_data_analysis.sql import to_snakecase

pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Allocation Prep

In [2]:
# Original allocation sheet without "None" Project IDs filtered out
# alloc_original = A1_data_prep.clean_allocation()

In [3]:
# alloc_original.shape

In [4]:
# prep_alloc = prep_allocation()

In [5]:
# len(prep_alloc)

In [6]:
# project_id_more_than_1 = ["19000021", "19000073", "19000120", "20000194"]

In [7]:
# alloc[alloc.allocation_project_id.isin(project_id_more_than_1)]

In [8]:
# alloc = A7_accounting_analysis.final_allocation()

In [9]:
# check to see if project ID match across all the original vs. grouped allocation data sets
# project_id_merged = set(alloc.allocation_project_id.unique().tolist())
# project_id_og = set(prep_alloc.allocation_project_id.unique().tolist())
# project_id_og - project_id_merged
# project_id_merged - project_id_og

In [10]:
# len(alloc) == len(prep_alloc)

### Expenditures Prep 

In [11]:
expenditure = A7_accounting_analysis.final_expenditures()

In [12]:
expenditure._merge.value_counts()

right_only    80
both          52
left_only     40
Name: _merge, dtype: int64

#### Double check project ids w/ original dataframe before groupby

In [13]:
og_expenditure = pd.read_excel(
    f"{A1_data_prep.GCS_FILE_PATH}3010_3020 Expenditure for fund 0046.xls",
    sheet_name="Download",
)

In [14]:
og_expenditure = og_expenditure.iloc[9:].reset_index(drop=True)

In [15]:
og_expenditure.columns = og_expenditure.iloc[0]

In [16]:
og_expenditure = og_expenditure.drop(og_expenditure.index[0]).reset_index(drop=True)

In [17]:
og_expenditure = (og_expenditure.loc[og_expenditure["FY"] > "2017"]).reset_index()

In [18]:
og_expenditure.FY.value_counts()

2022    279
2021    222
2020    208
2023    189
2018    180
2019    172
Name: FY, dtype: int64

In [19]:
og_expenditure = og_expenditure[
    og_expenditure["Appr Unit"].str.contains("101")
    | og_expenditure["Appr Unit"].str.contains("301R")
]

In [20]:
og_expenditure = A7_accounting_analysis.clean_project_ids(og_expenditure, "Project")

In [21]:
len(expenditure), expenditure.project.nunique(), len(
    og_expenditure
), og_expenditure.Project.nunique()

(172, 172, 423, 172)

In [22]:
# check to see if 2 columns are the same
project_id_grouped = set(expenditure.project.unique().tolist())
project_id_og = set(og_expenditure.Project.unique().tolist())
project_id_og - project_id_grouped

set()

### Project Status Prep

In [23]:
project_status = A7_accounting_analysis.prep_project_status()

In [24]:
project_status.project.nunique()

52

#### Double check project ids w/ original dataframe before groupby

In [25]:
project_status_og = pd.read_excel(
    f"{A1_data_prep.GCS_FILE_PATH}Project status for fund 0046 as of 12-5-22.xls",
    sheet_name="Download",
)

In [26]:
# First few rows are not data
project_status_og = project_status_og.iloc[6:].reset_index(drop=True)

# Cast first row as column names
project_status_og.columns = project_status_og.iloc[0]

# Drop the first row as they are now column names
project_status_og = project_status_og.drop(project_status_og.index[0]).reset_index(
    drop=True
)

In [27]:
project_status_og.Project.nunique()

52

In [28]:
# m1_function = merge_allocation_expenditures()

In [29]:
# len(alloc), len(m1_function)

In [30]:
m2 = A7_accounting_analysis.merge2_project_status()

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [31]:
(m2.isna().sum()) / len(m2)

allocation_project_id             0.00
allocation_ppno                   0.00
allocation_ea                     0.00
allocation_project_#              0.00
allocation_components             0.00
allocation_award_year             0.00
allocation_phase                  0.00
allocation_sb1_funding            0.00
allocation_ggrf_funding           0.00
Sum of Allocation Amount          0.00
project_project_title             0.00
301R_tot_exp                      0.00
101_tot_exp                       0.00
Allocation_Expenditure_Merge      0.00
billed                            0.74
reimbursements                    0.74
Allocation_Project_Status_Merge   0.00
dtype: float64

## Final Clean Up

In [32]:
preview_cols = ["Award Year",
    "Title",
    "Project ID",
    "Ea",
    "Comments",
    "Sum of Sb1 Funding",
    "Sum of GGRF Funding",
    "Sum Of Allocation Amount",
    "Sum of GGRF (0046, xx301R expenditure)",
    "Sum of SB1 (0046, xx101 Expenditure)",
    "Sum of Expenditure",
    "Sum Billed",
    "Sum of Reimbursements",
    "Remaining Allocation",
    "Allocation Expenditure Merge",
    "Allocation Project Status Merge"]

In [33]:
accounting_analysis, aggregated = A7_accounting_analysis.final_accounting_analysis()

In [34]:
# accounting_analysis['Project ID'].nunique() == alloc.allocation_project_id.nunique()

In [35]:
# accounting_analysis[preview_cols].loc[accounting_analysis.Comments == "More than $400k: need to correct funding"]

In [36]:
# accounting_analysis.loc[accounting_analysis["Project ID"] == "22000241"]